In [1]:
import argparse
import time
import msgpack
from enum import Enum, auto

import numpy as np
import networkx as nx

from udacidrone import Drone
from udacidrone.connection import MavlinkConnection
from udacidrone.messaging import MsgID
from udacidrone.frame_utils import global_to_local

from planning_utils import create_grid
from udacity_stuff import extract_polygons, load_data

from spejson import prune_graph, create_graph, sample_points, calculate_offset, prune_path, latlon2loc

In [2]:
class States(Enum):
    MANUAL = auto()
    ARMING = auto()
    TAKEOFF = auto()
    WAYPOINT = auto()
    LANDING = auto()
    DISARMING = auto()
    PLANNING = auto()

In [3]:
class MotionPlanning(Drone):

    def __init__(self, connection):
        super().__init__(connection)

        self.target_position = np.array([0.0, 0.0, 0.0])
        self.waypoints = []
        self.in_mission = True
        self.check_state = {}

        # initial state
        self.flight_state = States.MANUAL

        # register all your callbacks here
        self.register_callback(MsgID.LOCAL_POSITION, self.local_position_callback)
        self.register_callback(MsgID.LOCAL_VELOCITY, self.velocity_callback)
        self.register_callback(MsgID.STATE, self.state_callback)

    def local_position_callback(self):
        if self.flight_state == States.TAKEOFF:
            if -1.0 * self.local_position[2] > 0.95 * self.target_position[2]:
                self.waypoint_transition()
        elif self.flight_state == States.WAYPOINT:
            if np.linalg.norm(self.target_position[0:2] - self.local_position[0:2]) < 1.0:
                if len(self.waypoints) > 0:
                    self.waypoint_transition()
                else:
                    if np.linalg.norm(self.local_velocity[0:2]) < 1.0:
                        self.landing_transition()

    def velocity_callback(self):
        if self.flight_state == States.LANDING:
            if self.global_position[2] - self.global_home[2] < 0.1:
                if abs(self.local_position[2]) < 0.01:
                    self.disarming_transition()

    def state_callback(self):
        if self.in_mission:
            if self.flight_state == States.MANUAL:
                self.arming_transition()
            elif self.flight_state == States.ARMING:
                if self.armed:
                    self.plan_path()
            elif self.flight_state == States.PLANNING:
                self.takeoff_transition()
            elif self.flight_state == States.DISARMING:
                if ~self.armed & ~self.guided:
                    self.manual_transition()

    def arming_transition(self):
        self.flight_state = States.ARMING
        print("arming transition")
        self.arm()
        self.take_control()

    def takeoff_transition(self):
        self.flight_state = States.TAKEOFF
        print("takeoff transition")
        self.takeoff(self.target_position[2])

    def waypoint_transition(self):
        self.flight_state = States.WAYPOINT
        print("waypoint transition")
        self.target_position = self.waypoints.pop(0)
        print('target position', self.target_position)
        self.cmd_position(self.target_position[0], self.target_position[1], self.target_position[2], self.target_position[3])

    def landing_transition(self):
        self.flight_state = States.LANDING
        print("landing transition")
        self.land()

    def disarming_transition(self):
        self.flight_state = States.DISARMING
        print("disarm transition")
        self.disarm()
        self.release_control()

    def manual_transition(self):
        self.flight_state = States.MANUAL
        print("manual transition")
        self.stop()
        self.in_mission = False

    def send_waypoints(self):
        print("Sending waypoints to simulator ...")
        data = msgpack.dumps(self.waypoints)
        self.connection._master.write(data)

    def plan_path(self):
        self.flight_state = States.PLANNING
        print("Searching for a path ...")
        TARGET_ALTITUDE = 5
        SAFETY_DISTANCE = 5

        self.target_position[2] = TARGET_ALTITUDE

        # TODO: read lat0, lon0 from colliders into floating point values
        
        # TODO: set home position to (lon0, lat0, 0)

        # TODO: retrieve current global position
 
        # TODO: convert to current local position using global_to_local()
        
        print('global home {0}, position {1}, local position {2}'.format(self.global_home, self.global_position,
                                                                         self.local_position))
        # Read in obstacle map
        data = np.loadtxt('colliders.csv', delimiter=',', dtype='Float64', skiprows=2)
        
        # Define a grid for a particular altitude and safety margin around obstacles
        grid, north_offset, east_offset = create_grid(data, TARGET_ALTITUDE, SAFETY_DISTANCE)
        print("North offset = {0}, east offset = {1}".format(north_offset, east_offset))
        # Define starting point on the grid (this is just grid center)
        grid_start = (-north_offset, -east_offset)
        # TODO: convert start position to current position rather than map center
        
        # Set goal as some arbitrary position on the grid
        grid_goal = (-north_offset + 10, -east_offset + 10)
        # TODO: adapt to set goal as latitude / longitude position and convert

        # Run A* to find a path from start to goal
        # TODO: add diagonal motions with a cost of sqrt(2) to your A* implementation
        # or move to a different search space such as a graph (not done here)
        print('Local Start and Goal: ', grid_start, grid_goal)
        path, _ = a_star(grid, heuristic, grid_start, grid_goal)
        
        # TODO: prune path to minimize number of waypoints
        # TODO (if you're feeling ambitious): Try a different approach altogether!

        # Convert path to waypoints
        waypoints = [[p[0] + north_offset, p[1] + east_offset, TARGET_ALTITUDE, 0] for p in path]
        # Set self.waypoints
        self.waypoints = waypoints
        # TODO: send waypoints to sim
        self.send_waypoints()

    def start(self):
        self.start_log("Logs", "NavLog.txt")

        print("starting connection")
        self.connection.start()

        # Only required if they do threaded
        # while self.in_mission:
        #    pass

        self.stop_log()


In [4]:
print("Loading data...")
data = load_data('colliders.csv')
print("Loaded!")

# get obstacles
print("Extracting polygons...")
obstacles = extract_polygons(data)

# Sample points randomly 
print("Sampling points...")
np.random.seed(420)
points = sample_points(data, obstacles, 1000, z_range=(10,15))

print("Building graph...")
# build a graph with a branching factor of 5
graph, kd_tree = create_graph(points, bf=5, return_tree=True)

print("Pruning graph...")
graph = prune_graph(graph, obstacles, copy_graph=True)

Loading data...
Loaded!
Extracting polygons...
Sampling points...
Building graph...
Pruning graph...


In [8]:
def calculate_path(graph, kd_tree, points, start, goal, prune=False):

    # find nodes closest to start and goal
    (d1,d2),(s_idx,g_idx) = kd_tree.query([start, goal])

    start_node = tuple(*points[s_idx])
    goal_node = tuple(*points[g_idx])

    try:
        print("Searching for the path...")
        path = nx.algorithms.shortest_paths.astar.astar_path(graph, start_node, goal_node)
        print("Pruning the path...")
    except nx.exception.NetworkXNoPath:
        print("No valid path found. Try with a more dense graph")
        exit(-1)
        
    if prune:
        path = prune_path(path, obstacles)

    # append path with attitude
    path = [(int(p[0]), int(p[1]), int(p[2]), 0) for p in path]
    
    return path

In [12]:
class MP(Drone):

    def __init__(self, connection, environment_graph, goal_global):
        super().__init__(connection)
        
        self.target_position = np.array([0.0, 0.0, 0.0])
        self.waypoints = []
        self.in_mission = True
        self.check_state = {}
        self.graph = environment_graph
        self.goal_global = goal_global

        # initial state
        self.flight_state = States.MANUAL

        # register all your callbacks here
        self.register_callback(MsgID.LOCAL_POSITION, self.local_position_callback)
        self.register_callback(MsgID.LOCAL_VELOCITY, self.velocity_callback)
        self.register_callback(MsgID.STATE, self.state_callback)

    def local_position_callback(self):
        print("position cb")
        if self.flight_state == States.TAKEOFF:
            if -1.0 * self.local_position[2] > 0.95 * self.target_position[2]:
                print(-1.0 * self.local_position[2])
                print(0.95 * self.target_position[2])
                self.waypoint_transition()
        elif self.flight_state == States.WAYPOINT:
            if np.linalg.norm(self.target_position[0:2] - self.local_position[0:2]) < 2.0:
                if len(self.waypoints) > 0:
                    self.waypoint_transition()
                else:
                    # if no waypoints left
                    if np.linalg.norm(self.local_velocity[0:2]) < 1.0:
                        self.landing_transition()

    def velocity_callback(self):
        print("velocity cb")
        if self.flight_state == States.LANDING:
            if self.global_position[2] - self.global_home[2] < 0.1:
                if abs(self.local_position[2]) < 0.01:
                    self.disarming_transition()

    def state_callback(self):
        print("state cb")
        if self.in_mission:
            if self.flight_state == States.MANUAL:
                self.arming_transition()
            elif self.flight_state == States.ARMING:
                if self.armed:
                    self.plan_path()
            elif self.flight_state == States.PLANNING:
                self.takeoff_transition()
            elif self.flight_state == States.DISARMING:
                if ~self.armed & ~self.guided:
                    self.manual_transition()

    def arming_transition(self):
        print("arming transition")
        self.flight_state = States.ARMING
        self.arm()
        self.take_control()

    def takeoff_transition(self):
        print("takeoff transition")
        self.flight_state = States.TAKEOFF
        self.takeoff(self.target_position[2])

    def waypoint_transition(self):
        print("waypoint transition")
        self.flight_state = States.WAYPOINT
        self.target_position = self.waypoints.pop(0)
        print('target position', self.target_position)
        self.cmd_position(*self.target_position)

    def landing_transition(self):
        self.flight_state = States.LANDING
        print("landing transition")
        self.land()

    def disarming_transition(self):
        self.flight_state = States.DISARMING
        print("disarm transition")
        self.disarm()
        self.release_control()

    def manual_transition(self):
        self.flight_state = States.MANUAL
        print("manual transition")
        self.stop()
        self.in_mission = False

    def send_waypoints(self):
        print("Sending waypoints to simulator ...")
        data = msgpack.dumps(self.waypoints)
        self.connection._master.write(data)

    def plan_path(self):
        elf.flight_state = States.PLANNING
        self.set_home_position(*self.global_position)
        
        self.goal_local = latlon2loc(goal_global,
                             anchor_global=(self.global_home[1],
                                            self.global_home[0],
                                            self.global_home[2]))
        print("My goal is:")
        print(self.goal_local)
        
        print("Calculating path...")
        path = calculate_path(graph,
                              kd_tree,
                              points,
                              start=(0,0,0),
                              goal=goal_local,
                              prune=False)
        
        
        print("Sending waypoints...")
        self.waypoints = path
        self.target_position = path[0]
        self.send_waypoints()

    def start(self):
        self.start_log("Logs", "NavLog.txt")
        print("starting connection")
        self.connection.start()
        self.stop_log()

In [9]:
goal_global = (37.7951484,-122.393686, 10)

# goal_local = latlon2loc(goal_global, anchor=(self.global_home[1], self.global_home[0], self.global_home[2]))

goal_local = (400,400,10)

path = calculate_path(graph,
                      kd_tree,
                      points,
                      start=(0,0,5),
                      goal=goal_local,
                      prune=False)

Searching for the path...
Pruning the path...


In [7]:
graph

NameError: name 'graph' is not defined

In [4]:
conn = MavlinkConnection('tcp:{0}:{1}'.format('127.0.0.1', 5760), timeout=60)
drone = MotionPlanning(conn)

Logs/TLog.txt


In [6]:
drone.arm()
time.sleep(5)
drone.disarm()

In [7]:
drone = MP(conn, graph, goal_global=(37.7951484,-122.393686, 10))
drone.arm()
time.sleep(5)
drone.disarm()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tomek/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-19831cc4b903>", line 1, in <module>
    drone = MP(conn, graph, goal_global=(37.7951484,-122.393686, 10))
NameError: name 'MP' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tomek/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tomek/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/

NameError: name 'MP' is not defined

In [ ]:
!ls

In [ ]:
drone = MP(conn, graph, goal_global=(37.7951484,-122.393686, 10))
drone.start()